In [1]:
from __future__ import print_function
from pyspark.sql import SparkSession
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import *
from pyspark.sql.functions import split

In [2]:
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

In [3]:
df = spark.read.format('csv').option("escape","\"").option('header',True).load("s3://msbx5420-2020/yongbo/news.csv")

In [4]:
df.printSchema

<bound method DataFrame.printSchema of DataFrame[index: string, authors: string, title: string, publish_date: string, description: string, text: string, url: string]>

In [5]:
df.collect()[0]

Row(index='0', authors="['Cbc News']", title="Coronavirus a 'wake-up call' for Canada's prescription drug supply", publish_date='2020-03-27 08:00:00', description='Canadian pharmacies are limiting how much medication can be dispensed to try to prevent shortages, recognizing that most active ingredients for drugs come from India and China and medical supply chains have been disrupted by the spread of COVID-19.', text='Canadian pharmacies are limiting how much medication can be dispensed to try to prevent shortages, recognizing that most active ingredients for drugs come from India and China and medical supply chains have been disrupted by the spread of COVID-19. Provincial regulatory colleges are complying with the Canadian Pharmacists Association call to limit the amount of medications given to patients to 30-day\xa0supplies. The goal is to stop people from refilling prescriptions early and to ensure life-saving drugs don\'t run short when supply chains are\xa0vulnerable. Mina Tadrous 

In [6]:
newdf = df.withColumn('text', regexp_replace('text', '\xa0', ''))
newdf = newdf.withColumn('text', regexp_replace('text', '\\\'', ''))

In [7]:
newdf.take(1)

[Row(index='0', authors="['Cbc News']", title="Coronavirus a 'wake-up call' for Canada's prescription drug supply", publish_date='2020-03-27 08:00:00', description='Canadian pharmacies are limiting how much medication can be dispensed to try to prevent shortages, recognizing that most active ingredients for drugs come from India and China and medical supply chains have been disrupted by the spread of COVID-19.', text='Canadian pharmacies are limiting how much medication can be dispensed to try to prevent shortages, recognizing that most active ingredients for drugs come from India and China and medical supply chains have been disrupted by the spread of COVID-19. Provincial regulatory colleges are complying with the Canadian Pharmacists Association call to limit the amount of medications given to patients to 30-daysupplies. The goal is to stop people from refilling prescriptions early and to ensure life-saving drugs dont run short when supply chains arevulnerable. Mina Tadrous is a phar

In [28]:
rdd_text = newdf.select('text').rdd

In [16]:
counts_rdd = rdd_text.flatMap(lambda x: x[0].split(" ")).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)

In [17]:
type(counts_rdd)

pyspark.rdd.PipelinedRDD

In [21]:
counts_rdd = counts_rdd.toDF('word','count')

In [22]:
counts_rdd.collect()

[Row(word='has', count=11170),
 Row(word='written', count=110),
 Row(word='an', count=7809),
 Row(word='to', count=82495),
 Row(word='on', count=21124),
 Row(word='of', count=60065),
 Row(word='Day', count=193),
 Row(word='annual', count=181),
 Row(word='street', count=130),
 Row(word='leaders', count=324),
 Row(word='Dave', count=37),
 Row(word='Jaworsky,', count=1),
 Row(word='Chief', count=473),
 Row(word='Bryan', count=25),
 Row(word='Dr.', count=2408),
 Row(word='is', count=24348),
 Row(word='true', count=97),
 Row(word='transmit', count=37),
 Row(word='virus', count=3126),
 Row(word='have', count=16525),
 Row(word='underlying', count=143),
 Row(word='The', count=14716),
 Row(word='groups', count=364),
 Row(word='people', count=10510),
 Row(word='gathered', count=55),
 Row(word='was', count=8222),
 Row(word='police.', count=13),
 Row(word='Regional', count=205),
 Row(word='said', count=18059),
 Row(word='"some', count=16),
 Row(word='pre-St.', count=1),
 Row(word='hundred', count=

In [23]:
type(counts_rdd)

pyspark.sql.dataframe.DataFrame

In [24]:
print((counts_rdd.count(), len(counts_rdd.columns)))

(110744, 2)


In [27]:
counts_rdd.orderBy(desc("count")).collect()

[Row(word='the', count=113649),
 Row(word='to', count=82495),
 Row(word='of', count=60065),
 Row(word='and', count=55072),
 Row(word='in', count=48456),
 Row(word='a', count=42250),
 Row(word='for', count=25529),
 Row(word='is', count=24348),
 Row(word='that', count=21667),
 Row(word='on', count=21124),
 Row(word='are', count=20494),
 Row(word='said', count=18059),
 Row(word='have', count=16525),
 Row(word='be', count=15575),
 Row(word='with', count=15557),
 Row(word='The', count=14716),
 Row(word='at', count=14274),
 Row(word='as', count=13050),
 Row(word='from', count=12857),
 Row(word='will', count=12065),
 Row(word='', count=11827),
 Row(word='has', count=11170),
 Row(word='people', count=10510),
 Row(word='COVID-19', count=9785),
 Row(word='not', count=9313),
 Row(word='who', count=9012),
 Row(word='it', count=9008),
 Row(word='said.', count=8404),
 Row(word='been', count=8330),
 Row(word='or', count=8303),
 Row(word='was', count=8222),
 Row(word='health', count=8201),
 Row(word='

In [32]:
counts_rdd.orderBy(desc("count")).show()

+----+------+
|word| count|
+----+------+
| the|113649|
|  to| 82495|
|  of| 60065|
| and| 55072|
|  in| 48456|
|   a| 42250|
| for| 25529|
|  is| 24348|
|that| 21667|
|  on| 21124|
| are| 20494|
|said| 18059|
|have| 16525|
|  be| 15575|
|with| 15557|
| The| 14716|
|  at| 14274|
|  as| 13050|
|from| 12857|
|will| 12065|
+----+------+
only showing top 20 rows



word count with Dataframe (we all know it's SLOWER Version!!!)

In [30]:
import pyspark.sql.functions as f

In [31]:
newdf.withColumn('word', f.explode(f.split(f.col('text'), ' '))).groupBy('word').count()\
    .sort('count', ascending=False)\
    .show()

+----+------+
|word| count|
+----+------+
| the|113649|
|  to| 82495|
|  of| 60065|
| and| 55072|
|  in| 48456|
|   a| 42250|
| for| 25529|
|  is| 24348|
|that| 21667|
|  on| 21124|
| are| 20494|
|said| 18059|
|have| 16525|
|  be| 15575|
|with| 15557|
| The| 14716|
|  at| 14274|
|  as| 13050|
|from| 12857|
|will| 12065|
+----+------+
only showing top 20 rows



of course, the results match!!!

Interesting experiment for fun, check number of words in each news (or you can say "each row")

In [35]:
nnewdf = newdf.withColumn('wordCount', f.size(f.split(f.col('text'), ' ')))
nnewdf.show()

+-----+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+---------+
|index|             authors|               title|       publish_date|         description|                text|                 url|wordCount|
+-----+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+---------+
|    0|        ['Cbc News']|Coronavirus a 'wa...|2020-03-27 08:00:00|Canadian pharmaci...|Canadian pharmaci...|https://www.cbc.c...|      395|
|    1|        ['Cbc News']|Yukon gov't names...|2020-03-27 01:45:00|The Yukon governm...|The Yukon governm...|https://www.cbc.c...|      303|
|    2|['The Associated ...|U.S. Senate passe...|2020-03-26 05:13:00|The Senate has pa...|The Senate late W...|https://www.cbc.c...|      974|
|    3|        ['Cbc News']|Coronavirus: The ...|2020-03-27 00:36:00|Scientists around...|Scientists around...|https://www.cbc.c...|      721|

In [36]:
counts_rdd.write.parquet('s3://msbx5420-2020/yongbo/counts.parquet')

In [37]:
nnewdf.write.parquet('s3://msbx5420-2020/yongbo/news_countnews.parquet')